In [1]:
import pandas as pd
import json

In [18]:

### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

## From scrapy spider

In [2]:
json_file = json.load(open(r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\dailyCases.json', 'r'))

In [3]:
range(len(json_file))

range(0, 28)

In [4]:
# w1= json_file[0]
# w1[0]['data']

In [5]:
json_file[0]
data = []

for date in json_file:
    for x in range(0, 10):
        data1 = (date[0]['data'][x] + date[1]['data'][x])
        data.append(data1)
# data[:2]

In [6]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-01.php and ...-02.php

list_1 = [
    'No', 'Kabupaten / County', 'New PPLN / PMI', 'New PPLN / PMI', 'new PPDN', 'total PPDN',	'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'date'
]

list_2 = ['No', 'Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'date']

list_column_names = list_1 + list_2
# print(list_column_names)

In [7]:
df = pd.DataFrame(data, columns= list_column_names)
df.head()

,No,Kabupaten / County,New PPLN / PMI,New PPLN / PMI,new PPDN,total PPDN,new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,...,Kabupaten / County,new_treatment,total_treatment,new_recovered,total_recovered,new_deaths,total_deaths,new_cases,total_cases,date
0,1,JEMBRANA,0,26,0,11,1,626,0,0,...,JEMBRANA,-7,70,7,579,1,14,1,663,05 Dec 2020
1,2,TABANAN,0,21,2,10,23,1466,0,0,...,TABANAN,-10,434,35,1016,0,47,25,1497,05 Dec 2020
2,3,BADUNG,0,27,0,4,18,2478,0,0,...,BADUNG,-9,159,27,2299,0,51,18,2509,05 Dec 2020
3,4,DENPASAR,0,48,-1,22,28,3891,0,0,...,DENPASAR,1,166,26,3705,0,90,27,3961,05 Dec 2020
4,5,GIANYAR,0,29,-1,10,16,1951,0,0,...,GIANYAR,-16,234,31,1681,0,75,15,1990,05 Dec 2020


In [8]:
# # Check dataframe
# df.columns
# df.describe()
# df.dtypes

In [9]:
# delete duplicate columns and convert dtypes
df = df.loc[:,~df.columns.duplicated()]

df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df['Name_EN'] = df['Name_EN'].str.casefold()
# df.dtypes

In [10]:
from dateutil.parser import parse

df['Date'] = df['date'].apply(lambda x: parse(x))
df.Date[1]

Timestamp('2020-12-05 00:00:00')

In [11]:
# data cleaning, processing
#------------------------

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])
df_ref['Name_EN'] = df_ref['Name_Indo'].str.casefold()
df_ref.replace('kota denpasar', 'denpasar', inplace=True)
df_ref.replace('totals', 'bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

# df_ref.head(15)

In [12]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')

# convert to integers
int_numbers = ['total_recovered','new_deaths','total_deaths','new_total','total_cases']

df_merged[['total_recovered','new_deaths','total_deaths','new_total','total_cases']] = df_merged[['total_recovered','new_deaths','total_deaths','new_cases','total_cases']].apply(pd.to_numeric)

## add columns
df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100000
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015'] * 100000
df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015']
df_merged['cases7_per_100k'] = df_merged['cases7_per_100k'].apply(pd.to_numeric).round(2)

df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015']
df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'].rolling(window=7, center=False).mean()
df_merged['deaths7_per_100k'] = df_merged['deaths7_per_100k'].apply(pd.to_numeric).round(2)



In [13]:
## drop temporary unnecessary columns
drop_columns = ['New PPLN / PMI', 'new PPDN', 'total PPDN',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'Capital', 'population_2000',
       'population_2010', 'HDI_2014_estimate', 'Gouvernor',
       'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'area_km2' ]
df_clean = df_merged.drop(columns=drop_columns)

In [14]:
## check NaN in df 
# nan_rows = df_clean[df_clean['population_2015'].isnull()]
# nan_rows

In [15]:
# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19

df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)
# df_merged['CFR']


In [16]:
## add id from geojson
geojson_bali = 'new_bali_geojson_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map
## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
# df_merged.head(15)

In [17]:
df_merged.to_csv('bali_regency_data.csv')

In [18]:

### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset



In [19]:
# 1. connect to kaggle and get latest dataset

#--------------------------
## RUN IN CONDA BASE environment 
# -----------------------------------

# (kaggle api installed here)


In [20]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [21]:
from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [23]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')

df_indo.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,20,39,...,106.836118,-6.204699,0.18,45.09,0.0,1.84,4.09%,7.98%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,20,39,...,106.836118,-6.204699,0.18,45.27,0.0,1.84,4.07%,7.94%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,120,...,107.603708,-6.920432,0.07,0.27,0.0,0.11,41.67%,1000.00%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,1,...,101.805109,0.511648,0.16,0.33,0.0,0.16,50.00%,50.00%,NaN,NaN


In [24]:
# df_indo.dtypes

In [25]:
print(list(df_merged.columns))

['No', 'Kabupaten / County', 'New PPLN / PMI', 'new PPDN', 'total PPDN', 'new_local_transmission', 'total_local_transmission', 'new_other_transmission', 'total_other_transmission', 'new_cases', 'total_cases', 'date', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'Name_EN', 'Date', 'Name_Indo', 'Capital', 'area_km2', 'population_2000', 'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor', 'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'new_total', 'total_cases_per_100k', 'total_deaths_per_100k', 'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR', 'id']


In [26]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [27]:

# add columns so consistent with Bali_regency data

df_indo['Name_EN'] = df_indo['Location'].str.casefold()
df_indo['new_cases_per_100k'] = df_indo['New Cases per Million'] / 100
df_indo['new_deaths_per_100k'] = df_indo['New Deaths per Million'] / 100

df_indo['total_cases_per_100k'] = df_indo['Total Cases per Million'] / 100
df_indo['total_deaths_per_100k'] = df_indo['Total Deaths per Million'] / 100

# rolling 7-day average
df_indo['cases7'] = df_indo['new_cases'].rolling(window=7, center=False).mean().round(2)
df_indo['deaths7'] = df_indo['new_deaths'].rolling(window=7, center = False).mean().round(2)

df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths', 'Name_EN', 'new_cases_per_100k',
       'new_deaths_per_100k', 'total_cases_per_100k', 'total_deaths_per_100k',
       'cases7', 'deaths7'],
      dtype='object')

In [28]:
## add id from geojson
geojson_indo = 'indo_level1_id.geojson'

# open new geojson (with id) file
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [29]:
indo_geo_id['features'][0]['id']

'1'

In [30]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['ADM1_EN'])

list(set(names_df) - set(provinces))

['Indonesia', 'DKI Jakarta']

In [31]:
# adjust dki jakarta
df_indo = df_indo.replace("DKI Jakarta", 'Dki Jakarta')

In [32]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['ADM1_EN'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['ADM1_EN']] = province['id']
indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.head()


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,Growth Factor of New Cases,Growth Factor of New Deaths,Name_EN,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,cases7,deaths7,id
0,3/1/2020,ID-JK,Dki Jakarta,2,0,0,2,489,20,39,...,NaN,NaN,dki jakarta,0.0018,0.0,0.4509,0.0184,NaN,NaN,6
1,3/2/2020,ID-JK,Dki Jakarta,2,0,0,2,491,20,39,...,1.0,1.0,dki jakarta,0.0018,0.0,0.4527,0.0184,NaN,NaN,6
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,NaN,NaN,indonesia,0.0001,0.0,0.0001,0.0000,NaN,NaN,0
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,120,...,NaN,NaN,jawa barat,0.0007,0.0,0.0027,0.0011,NaN,NaN,9
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,1,...,NaN,NaN,riau,0.0016,0.0,0.0033,0.0016,NaN,NaN,26


In [33]:
df_indo.to_csv('indo_province_data.csv')